# Question - 6
Suppose there are two slot machines that you can play (we will call them “Machine A” and “Machine B”). Machine A will pay you $0.50 each time you play it. Machine B will pay you nothing with probability 9/10 and will pay you $20 with probability 1/10 each time you play it.

What is the policy that maximizes the expected total discounted reward, and what is the expected total discounted reward achieved by this policy? Assume a discount factor of 0.9.
1. Always Play A 
2. Always Play B
3. Alternate Play A & Play B

In [56]:
import numpy as np

P1 = np.array([
    [1, 0],
    [1, 0]
])

P2 = np.array([
    [0, 1],
    [0, 1]
])

P3 = np.array([
    [1/2, 1/2],
    [1/2, 1/2]
])
reward = np.array([0.5, 1/10*20])


discount_factor = 0.9

def compute_total_discounted_reward(policy, MAX_ITER=100000):
    V = np.array([0, 0])
    for i in range(0, MAX_ITER):
        V = reward + discount_factor*policy.dot(V)
    return V

print(
    compute_total_discounted_reward(P1)[0],
    compute_total_discounted_reward(P2)[1],
    compute_total_discounted_reward(P3).max(), sep="\n"
)

4.999999999999997
19.99999999999999
13.249999999999995


# Question - 8
Suppose there are two slot machines that you can play (we will call them “Machine A” and “Machine B”). Machine A will pay you $0.50 each time you play it. Machine B will pay you nothing with probability 9/10 and will pay you $20 with probability 1/10 each time you play it.

Suppose that you will play the slot machines described in Question 6 repeatedly, but now you do not know the payouts of the machines ahead of time. You will use the following algorithm, which is like a simplified version of Q-learning:

Maintain estimates of each machine’s payout, which we will call Q_A and Q_B. We will initially set these estimates to Q_A = Q_B = 0.
Play the machine with the greatest estimated payout. That is, if Q_A > Q_B then play Machine A. If Q_B > Q_A then play Machine B. If Q_A = Q_B, pick a machine to play at random.
Each time you play Machine i, use the payout p received on that play to update the estimated payout for the machine as Q_i := (1-a)*Q_i + a*p. 

What are the value states with a epsilon of 0.5 ?

In [39]:
import random

def policy(V, epsilon=0.5):
    if random.random() <= epsilon:
        return random.randint(0, 1)
    elif V[0] > V[1]:
        return 0
    else:
        return 1

def reward(slot):
    return 0.5 if slot == 0 else 2

Q = np.array([0.0, 0.0])
alpha = 0.98

for i in range(0, 10):
    slot = policy(Q)
    r = reward(slot)
    Q[slot] = (1-alpha)*Q[slot] + alpha*r

Q

array([0.4998, 2.    ])

# Question - 18

state   |   1   |   2   |   3
--------|-------|-------|------
1       |   1/4 |   1/2 |   1/4
2       |   1/3 |   0   |   2/3
3       |   1/2 |   0   |   1/2

Suppose that we receive a reward of 1 unit every time we visit state 2, and receive zero reward otherwise. If we start from state 1, what is the expected total discounted reward achieved with a discount factor of 0.9?

In [3]:
import numpy as np

P = np.array([
    [1/4, 1/2, 1/4],
    [1/3, 0, 2/3],
    [1/2, 0, 1/2]
])

R = np.array([0, 1, 0])

V = np.array([0, 0, 0])
discount_factor = 0.9

for i in range(0, 1000):
    V = R + discount_factor*P.dot(V)
    
print(V[0])

1.9148936170212754


# Question 12
You will roll a six-sided die. If the die lands on one, the game is over. Otherwise you have a choice: You can stop and I will pay you N dollars where N is the number that you rolled, or you can try again. You can retry as many times as you like until you decide to stop or you roll a one. 

What is the policy that maximizes the expected total discounted reward (assuming a discount factor of 0.9)?

In [158]:
import numpy as np
import random

states = np.array([0, 1, 2, 3, 4, 5])

rewards_roll = np.array([0, 0, 0, 0, 0, 0])
rewards_stop = np.array([0, 2, 3, 4, 5, 6])

ACTION_STOP = 0
ACTION_ROLL = 1
actions = np.array([ACTION_STOP, ACTION_ROLL])
P_stop = np.array([
    [1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 1]
])
P_roll = np.array([
    [1, 0, 0, 0, 0, 0],
    [1/6, 1/6, 1/6, 1/6, 1/6, 1/6],
    [1/6, 1/6, 1/6, 1/6, 1/6, 1/6],
    [1/6, 1/6, 1/6, 1/6, 1/6, 1/6],
    [1/6, 1/6, 1/6, 1/6, 1/6, 1/6],
    [1/6, 1/6, 1/6, 1/6, 1/6, 1/6]
])
discount_factor = 0.9

V = np.zeros(len(states))

def q_policy(state, Q, epsilon=0.01):
    if random.random() <= epsilon:
        return random.randint(ACTION_STOP, ACTION_ROLL)
    
    return ACTION_STOP if Q[state][ACTION_STOP] > Q[state][ACTION_ROLL] else ACTION_ROLL

Q = np.zeros((len(states), len(actions)))
Q[:,ACTION_STOP] =  rewards_stop + P_stop.dot(discount_factor*V)
Q[:,ACTION_ROLL] =  rewards_roll + P_roll.dot(discount_factor*V)

for i in range(0, 10000):
    for state in states:
        action = q_policy(state, Q)
        P = P_roll[state] if action == ACTION_ROLL else P_stop[state]
        r = rewards_roll[state] if action == ACTION_ROLL else rewards_stop[state]
        Q[state, action] = Q[state, action] + (r + P.dot(discount_factor*V))
    V = np.array([np.max(Q[s]) for s in states])
    

print('Q[state, action] ->\n', Q)

print('\nOptimal Policy: We stop if dice value is 4, 5, 6 otherwise roll')

Q[state, action] ->
 [[ 0.          0.        ]
 [30.92857143 32.14285714]
 [31.92857143 32.14285714]
 [40.         32.14285714]
 [50.         32.14285714]
 [60.         32.14285714]]

Optimal Policy: We stop if dice value is 4, 5, 6 otherwise roll
